# Project 6 :Bank Churn Prediction

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.show()
%matplotlib inline 
# To enable plotting graphs in Jupyter notebook
import seaborn as sns
from sklearn.linear_model import LinearRegression


# Remove scientific notations and display numbers with 2 decimal points instead
pd.options.display.float_format = '{:,.2f}'.format

In [4]:
#Load the file from local directory using pd.read_csv which is a special form of read_table
df = pd.read_csv("bank DATA.csv")
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,"101,348.88",1
1,2,15647311,Hill,608,Spain,Female,41,1,"83,807.86",1,0,1,"112,542.58",0
2,3,15619304,Onio,502,France,Female,42,8,"159,660.80",3,1,0,"113,931.57",1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,"93,826.63",0
4,5,15737888,Mitchell,850,Spain,Female,43,2,"125,510.82",1,1,1,"79,084.10",0


In [5]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
RowNumber,"10,000.00","5,000.50","2,886.90",1.00,"2,500.75","5,000.50","7,500.25","10,000.00"
CustomerId,"10,000.00","15,690,940.57","71,936.19","15,565,701.00","15,628,528.25","15,690,738.00","15,753,233.75","15,815,690.00"
CreditScore,"10,000.00",650.53,96.65,350.00,584.00,652.00,718.00,850.00
Age,"10,000.00",38.92,10.49,18.00,32.00,37.00,44.00,92.00
Tenure,"10,000.00",5.01,2.89,0.00,3.00,5.00,7.00,10.00
Balance,"10,000.00","76,485.89","62,397.41",0.00,0.00,"97,198.54","127,644.24","250,898.09"
NumOfProducts,"10,000.00",1.53,0.58,1.00,1.00,1.00,2.00,4.00
HasCrCard,"10,000.00",0.71,0.46,0.00,0.00,1.00,1.00,1.00
IsActiveMember,"10,000.00",0.52,0.50,0.00,0.00,1.00,1.00,1.00
EstimatedSalary,"10,000.00","100,090.24","57,510.49",11.58,"51,002.11","100,193.91","149,388.25","199,992.48"


In [6]:
df.drop('CustomerId', axis=1, inplace=True)

In [7]:
df.drop('RowNumber', axis=1, inplace=True)

In [8]:
df.drop('Surname', axis=1, inplace=True)

In [9]:
df = pd.get_dummies(df)
df = df.astype('float32')
df

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male
0,619.00,42.00,2.00,0.00,1.00,1.00,1.00,"101,348.88",1.00,1.00,0.00,0.00,1.00,0.00
1,608.00,41.00,1.00,"83,807.86",1.00,0.00,1.00,"112,542.58",0.00,0.00,0.00,1.00,1.00,0.00
2,502.00,42.00,8.00,"159,660.80",3.00,1.00,0.00,"113,931.57",1.00,1.00,0.00,0.00,1.00,0.00
3,699.00,39.00,1.00,0.00,2.00,0.00,0.00,"93,826.63",0.00,1.00,0.00,0.00,1.00,0.00
4,850.00,43.00,2.00,"125,510.82",1.00,1.00,1.00,"79,084.10",0.00,0.00,0.00,1.00,1.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771.00,39.00,5.00,0.00,2.00,1.00,0.00,"96,270.64",0.00,1.00,0.00,0.00,0.00,1.00
9996,516.00,35.00,10.00,"57,369.61",1.00,1.00,1.00,"101,699.77",0.00,1.00,0.00,0.00,0.00,1.00
9997,709.00,36.00,7.00,0.00,1.00,0.00,1.00,"42,085.58",1.00,1.00,0.00,0.00,1.00,0.00
9998,772.00,42.00,3.00,"75,075.31",2.00,1.00,0.00,"92,888.52",1.00,0.00,1.00,0.00,0.00,1.00


Main objective of this data analysis is to identify the features to determine the possibility of the customer to retained as a customer of the bank. So I'm keeping "Exited" column as the target and all others as the freatures. 

In [10]:
from sklearn.model_selection import train_test_split
X = df[['CreditScore','Age', 'Tenure', 'Balance', 'NumOfProducts',
       'HasCrCard', 'IsActiveMember', 'EstimatedSalary','Geography_France','Geography_Germany','Geography_Spain','Gender_Female','Gender_Male']]
Y = df['Exited']


X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state=1)

In [11]:
#Normalizing data
from sklearn import preprocessing
scaler = preprocessing.MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_train

array([[0.556     , 0.14864865, 0.3       , ..., 0.        , 0.        ,
        1.        ],
       [0.55200005, 0.14864865, 0.4       , ..., 0.        , 1.        ,
        0.        ],
       [0.52400005, 0.39189193, 0.6       , ..., 0.        , 1.        ,
        0.        ],
       ...,
       [0.64400005, 0.36486486, 0.90000004, ..., 0.        , 1.        ,
        0.        ],
       [0.62600005, 0.28378376, 0.8       , ..., 0.        , 1.        ,
        0.        ],
       [0.826     , 0.32432434, 0.6       , ..., 0.        , 0.        ,
        1.        ]], dtype=float32)

In [12]:
!pip install tensorflow==2.2.0

In [13]:
import tensorflow as tf

In [14]:
print(tf.__version__)

2.2.0


In [15]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Input, Dense

In [16]:
x_train=tf.convert_to_tensor(X_train)
y_train=tf.convert_to_tensor(y_train)
x_test=tf.convert_to_tensor(X_test)
y_test=tf.convert_to_tensor(y_test)

In [17]:
X_train.shape

(8000, 13)

In [18]:
# Initiate the model and add layers
nn_1 = tf.keras.models.Sequential([Input(shape=(X_train.shape[1],)),  # Input layer
                                   Dense(1)])                         # Output layer

# Add loss function, optimizer and evaluation metric to the model
nn_1.compile(loss = 'mse', 
             optimizer = 'sgd', 
             metrics = [ 'mae', 'mape']) # Many other metrics are avaialble. For classification: accuracy, Recall, Precision, FalseNegatives, etc.

# Train the model
nn_1.fit(X_train, y_train, epochs = 1000, verbose = 1)

Epoch 1/1000
250/250 [==============================] - 0s 1ms/step - loss: 0.2129 - mae: 0.3568 - mape: 197110688.0000
Epoch 2/1000
250/250 [==============================] - 0s 1ms/step - loss: 0.1664 - mae: 0.3151 - mape: 162257072.0000
Epoch 3/1000
250/250 [==============================] - 0s 1ms/step - loss: 0.1567 - mae: 0.3048 - mape: 152719232.0000
Epoch 4/1000
250/250 [==============================] - 0s 1ms/step - loss: 0.1517 - mae: 0.3019 - mape: 152619264.0000
Epoch 5/1000
250/250 [==============================] - ETA: 0s - loss: 0.1492 - mae: 0.3001 - mape: 151002656.00 - 0s 1ms/step - loss: 0.1486 - mae: 0.2988 - mape: 150458048.0000
Epoch 6/1000
250/250 [==============================] - 0s 1ms/step - loss: 0.1464 - mae: 0.2968 - mape: 149193920.0000
Epoch 7/1000
250/250 [==============================] - 0s 1ms/step - loss: 0.1448 - mae: 0.2948 - mape: 148005792.0000
Epoch 8/1000
250/250 [==============================] - 0s 1ms/step - loss: 0.1437 - mae: 0.2947 - m

250/250 [==============================] - 0s 1ms/step - loss: 0.1385 - mae: 0.2845 - mape: 146055456.0000
Epoch 68/1000
250/250 [==============================] - 0s 1ms/step - loss: 0.1384 - mae: 0.2851 - mape: 147018992.0000
Epoch 69/1000
250/250 [==============================] - 0s 1ms/step - loss: 0.1383 - mae: 0.2847 - mape: 146615040.0000
Epoch 70/1000
250/250 [==============================] - 0s 1ms/step - loss: 0.1384 - mae: 0.2848 - mape: 146351728.0000
Epoch 71/1000
250/250 [==============================] - 0s 1ms/step - loss: 0.1385 - mae: 0.2854 - mape: 147444544.0000
Epoch 72/1000
250/250 [==============================] - 0s 1ms/step - loss: 0.1384 - mae: 0.2843 - mape: 145941872.0000
Epoch 73/1000
250/250 [==============================] - 0s 1ms/step - loss: 0.1384 - mae: 0.2852 - mape: 147223040.0000
Epoch 74/1000
250/250 [==============================] - 0s 1ms/step - loss: 0.1384 - mae: 0.2849 - mape: 146580720.0000
Epoch 75/1000
250/250 [=======================

250/250 [==============================] - 0s 1ms/step - loss: 0.1383 - mae: 0.2843 - mape: 145840576.0000
Epoch 135/1000
250/250 [==============================] - 0s 1ms/step - loss: 0.1383 - mae: 0.2851 - mape: 147181088.0000
Epoch 136/1000
250/250 [==============================] - 0s 1ms/step - loss: 0.1383 - mae: 0.2845 - mape: 146228864.0000
Epoch 137/1000
250/250 [==============================] - 0s 1ms/step - loss: 0.1384 - mae: 0.2848 - mape: 146677552.0000
Epoch 138/1000
250/250 [==============================] - 0s 1ms/step - loss: 0.1383 - mae: 0.2849 - mape: 146758000.0000
Epoch 139/1000
250/250 [==============================] - 0s 1ms/step - loss: 0.1384 - mae: 0.2845 - mape: 146191872.0000
Epoch 140/1000
250/250 [==============================] - 0s 1ms/step - loss: 0.1384 - mae: 0.2856 - mape: 147731872.0000
Epoch 141/1000
250/250 [==============================] - 0s 1ms/step - loss: 0.1385 - mae: 0.2847 - mape: 146344592.0000
Epoch 142/1000
250/250 [===============

Epoch 200/1000
250/250 [==============================] - 0s 1ms/step - loss: 0.1384 - mae: 0.2850 - mape: 146925104.0000
Epoch 201/1000
250/250 [==============================] - 0s 1ms/step - loss: 0.1384 - mae: 0.2850 - mape: 146655264.0000
Epoch 202/1000
250/250 [==============================] - 0s 1ms/step - loss: 0.1385 - mae: 0.2851 - mape: 147051296.0000
Epoch 203/1000
250/250 [==============================] - 0s 1ms/step - loss: 0.1384 - mae: 0.2850 - mape: 147166768.0000
Epoch 204/1000
250/250 [==============================] - 0s 1ms/step - loss: 0.1383 - mae: 0.2846 - mape: 146471408.0000
Epoch 205/1000
250/250 [==============================] - 0s 1ms/step - loss: 0.1384 - mae: 0.2848 - mape: 146398112.0000
Epoch 206/1000
250/250 [==============================] - 0s 1ms/step - loss: 0.1384 - mae: 0.2850 - mape: 147047872.0000
Epoch 207/1000
250/250 [==============================] - 0s 1ms/step - loss: 0.1385 - mae: 0.2845 - mape: 146108144.0000
Epoch 208/1000
250/250 [

250/250 [==============================] - 0s 1ms/step - loss: 0.1384 - mae: 0.2847 - mape: 146263872.0000
Epoch 267/1000
250/250 [==============================] - 0s 1ms/step - loss: 0.1384 - mae: 0.2851 - mape: 147093792.0000
Epoch 268/1000
250/250 [==============================] - 0s 1ms/step - loss: 0.1384 - mae: 0.2845 - mape: 146357024.0000
Epoch 269/1000
250/250 [==============================] - 0s 1ms/step - loss: 0.1384 - mae: 0.2854 - mape: 147538528.0000
Epoch 270/1000
250/250 [==============================] - 0s 1ms/step - loss: 0.1383 - mae: 0.2846 - mape: 146340112.0000
Epoch 271/1000
250/250 [==============================] - 0s 1ms/step - loss: 0.1384 - mae: 0.2848 - mape: 146662672.0000
Epoch 272/1000
250/250 [==============================] - 0s 1ms/step - loss: 0.1383 - mae: 0.2846 - mape: 146266832.0000
Epoch 273/1000
250/250 [==============================] - 0s 1ms/step - loss: 0.1384 - mae: 0.2852 - mape: 147211168.0000
Epoch 274/1000
250/250 [===============

...

250/250 [==============================] - 0s 1ms/step - loss: 0.1385 - mae: 0.2849 - mape: 146580832.0000
Epoch 731/1000
250/250 [==============================] - 0s 1ms/step - loss: 0.1384 - mae: 0.2851 - mape: 147257840.0000
Epoch 732/1000
250/250 [==============================] - 0s 1ms/step - loss: 0.1384 - mae: 0.2844 - mape: 145997408.0000
Epoch 733/1000
250/250 [==============================] - 0s 1ms/step - loss: 0.1384 - mae: 0.2852 - mape: 146993408.0000
Epoch 734/1000
250/250 [==============================] - 0s 1ms/step - loss: 0.1383 - mae: 0.2851 - mape: 147406112.0000
Epoch 735/1000
250/250 [==============================] - 0s 1ms/step - loss: 0.1384 - mae: 0.2854 - mape: 147471792.0000
Epoch 736/1000
250/250 [==============================] - 0s 1ms/step - loss: 0.1384 - mae: 0.2840 - mape: 145296688.0000
Epoch 737/1000
250/250 [==============================] - 0s 1ms/step - loss: 0.1384 - mae: 0.2849 - mape: 146863904.0000
Epoch 738/1000
250/250 [===============

250/250 [==============================] - 0s 1ms/step - loss: 0.1384 - mae: 0.2847 - mape: 146159264.0000
Epoch 798/1000
250/250 [==============================] - 0s 1ms/step - loss: 0.1384 - mae: 0.2848 - mape: 146662448.0000
Epoch 799/1000
250/250 [==============================] - 0s 1ms/step - loss: 0.1385 - mae: 0.2856 - mape: 147630256.0000
Epoch 800/1000
250/250 [==============================] - 0s 1ms/step - loss: 0.1383 - mae: 0.2844 - mape: 146309088.0000
Epoch 801/1000
250/250 [==============================] - 0s 1ms/step - loss: 0.1385 - mae: 0.2855 - mape: 147567200.0000
Epoch 802/1000
250/250 [==============================] - 0s 1ms/step - loss: 0.1383 - mae: 0.2845 - mape: 146058400.0000
Epoch 803/1000
250/250 [==============================] - 0s 1ms/step - loss: 0.1384 - mae: 0.2849 - mape: 146595872.0000
Epoch 804/1000
250/250 [==============================] - 0s 1ms/step - loss: 0.1385 - mae: 0.2851 - mape: 147079296.0000
Epoch 805/1000
250/250 [===============

250/250 [==============================] - ETA: 0s - loss: 0.1380 - mae: 0.2845 - mape: 147013616.00 - 0s 1ms/step - loss: 0.1384 - mae: 0.2848 - mape: 146713504.0000
Epoch 864/1000
250/250 [==============================] - 0s 1ms/step - loss: 0.1384 - mae: 0.2842 - mape: 145698640.0000
Epoch 865/1000
250/250 [==============================] - 0s 1ms/step - loss: 0.1385 - mae: 0.2855 - mape: 147342752.0000
Epoch 866/1000
250/250 [==============================] - 0s 1ms/step - loss: 0.1384 - mae: 0.2844 - mape: 146096832.0000
Epoch 867/1000
250/250 [==============================] - 0s 1ms/step - loss: 0.1385 - mae: 0.2855 - mape: 147645856.0000
Epoch 868/1000
250/250 [==============================] - 0s 1ms/step - loss: 0.1384 - mae: 0.2845 - mape: 146400624.0000
Epoch 869/1000
250/250 [==============================] - 0s 2ms/step - loss: 0.1384 - mae: 0.2851 - mape: 147066160.0000
Epoch 870/1000
250/250 [==============================] - 0s 2ms/step - loss: 0.1384 - mae: 0.2845 - 

250/250 [==============================] - 0s 1ms/step - loss: 0.1384 - mae: 0.2854 - mape: 147710960.0000
Epoch 930/1000
250/250 [==============================] - 0s 1ms/step - loss: 0.1384 - mae: 0.2849 - mape: 146518368.0000
Epoch 931/1000
250/250 [==============================] - 0s 1ms/step - loss: 0.1384 - mae: 0.2846 - mape: 146428832.0000
Epoch 932/1000
250/250 [==============================] - 0s 1ms/step - loss: 0.1384 - mae: 0.2848 - mape: 146480240.0000
Epoch 933/1000
250/250 [==============================] - 0s 1ms/step - loss: 0.1385 - mae: 0.2848 - mape: 146939712.0000
Epoch 934/1000
250/250 [==============================] - 0s 1ms/step - loss: 0.1384 - mae: 0.2850 - mape: 146627776.0000
Epoch 935/1000
250/250 [==============================] - 0s 1ms/step - loss: 0.1383 - mae: 0.2837 - mape: 145349440.0000
Epoch 936/1000
250/250 [==============================] - 0s 1ms/step - loss: 0.1384 - mae: 0.2859 - mape: 148239072.0000
Epoch 937/1000
250/250 [===============

250/250 [==============================] - 0s 1ms/step - loss: 0.1385 - mae: 0.2851 - mape: 146964512.0000
Epoch 996/1000
250/250 [==============================] - 0s 1ms/step - loss: 0.1384 - mae: 0.2850 - mape: 147046704.0000
Epoch 997/1000
250/250 [==============================] - 0s 1ms/step - loss: 0.1384 - mae: 0.2849 - mape: 146748848.0000
Epoch 998/1000
250/250 [==============================] - 0s 2ms/step - loss: 0.1384 - mae: 0.2841 - mape: 145464944.0000
Epoch 999/1000
250/250 [==============================] - 0s 1ms/step - loss: 0.1384 - mae: 0.2853 - mape: 147688224.0000
Epoch 1000/1000
250/250 [==============================] - 0s 1ms/step - loss: 0.1383 - mae: 0.2847 - mape: 146545824.0000


# testing the model on the test data and predict on the test data

In [32]:
nn_1.evaluate(X_test, y_test)


63/63 [==============================] - 0s 1ms/step - loss: 0.1366 - mae: 0.2830 - mape: 143672432.0000


[0.13662441074848175, 0.28296008706092834, 143672432.0]

In [33]:
nn_1.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1)                 14        
Total params: 14
Trainable params: 14
Non-trainable params: 0
_________________________________________________________________


In [34]:
y_pred = nn_1.predict(X_test)
print(y_pred)

[[0.13960372]
 [0.1854027 ]
 [0.17435016]
 ...
 [0.0038441 ]
 [0.1771626 ]
 [0.3464248 ]]


In [35]:
# Confusion Matrix
from sklearn.metrics import confusion_matrix
cm1 = confusion_matrix(y_test, y_pred)
print(cm1)

ValueError: Classification metrics can't handle a mix of binary and continuous targets